In [1]:
import os
import json

import os.path as osp

import pandas as pd 

from collections import Counter

import numpy as np

from loguru import logger
from scanrefer_entity import ScanReferEntity
from nr3d_entity import NR3DEntity


os.chdir(osp.dirname(os.getcwd()))



In [2]:
a = ScanReferEntity('scanrefer','train',0.2,init_anno=True)

2022-11-15 13:49:22.205 | INFO     | data_util:get_scanrefer:72 -  len of train split : 36665
2022-11-15 13:49:22.252 | INFO     | scanrefer_entity:get_all_ann_by_scanid:58 -  length : 7191,ratio: 0.2


In [2]:
# from glob import glob
# from my_script.utils import readtxt
# consistency_list_qualitative_res_path = "/data3/xusc/exp/butd_detr/logs"

# all_data = {}

# for f in glob(consistency_list_qualitative_res_path+"/scanrefer20*" ):
#     data = np.loadtxt(f,delimiter='\n',dtype=np.str0).tolist()
#     name = f.split('/')[-1].split('.')[0]
#     print(len(data),name)
#     all_data[name] = data

In [3]:
# pick_samples = []
# for x in all_data['scanrefer20_all_consistenct_loss']:
#     if x not in  all_data['scanrefer20_no_constrastive_soft_token_consistency_loss_qualitative_list'] and \
#         x not in  all_data['scanrefer20_no_contrastive_consistency_loss_qualitative_list'] and \
#             x not in  all_data['scanrefer20_no_soft_token_consistency_loss_qualitative_list']:
#             pick_samples.append(x)

# len(pick_samples)
# np.savetxt(osp.join(consistency_list_qualitative_res_path,'only_all_consistency_works_qua_list.txt'),pick_samples,fmt="%s")

In [5]:









# for ratio in np.linspace(0.1,0.9,9):
#     logger.info(ratio)
#     NR3DEntity('nr3d','train',round(ratio,1))
#     logger.info("==================================================================")

# a = NR3DEntity('nr3d','train')
# a.split_labeled_according_assignment_id(0.5)


# for ratio in np.linspace(0.1,0.9,9):
#     logger.info(ratio)
#     SR3DEntity('sr3d','train',round(ratio,1))
    # logger.info("==================================================================")
# SR3DEntity('sr3d','train')



2022-11-15 12:54:00.240 | INFO     | nr3d_entity:get_split_list:21 -  length : 511
2022-11-15 12:54:04.366 | INFO     | nr3d_entity:get_all_ann_by_scanid:63 - annos num = 30037


2022-11-15 13:00:25.773 | INFO     | scanrefer_entity:get_ratio_split_list:35 -  length : 112
